# Trainer
千帆Python SDK 在使用[resource API实现发起训练微调](./api_based_finetune.ipynb)之外，还提供了Trainer API，可以更方便地实现一体化的训练微调pipeline。同时提供了状态事件回调函数的注册，通过事件分发实现训练流程状态事件的监控。


本例将基于qianfan==0.2.2展示通过Dataset加载本地数据集，并上传到千帆平台，基于ERNIE-Bot-turbo进行fine-tune，并使用Model进行批量跑评估数据，直到最终完成服务发布，并最终实现服务调用的完整过程。

In [ ]:
! pip install "qianfan>=0.2.8" -U

In [1]:
import qianfan
qianfan.__version__

'0.2.7'

## 前置准备
- 初始化千帆安全认证AK、SK

In [2]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

#### 导入依赖
- `qianfan.trainer.consts` trainer使用中所用到的常量
- `qianfan.resources.console.consts` api层面定义的字段常量
- `qianfan.trainer.configs` trainer使用所需要的config配置数据类
- `qianfan.resources.QfMessages` 用于组装qianfan.ChatCompletion的输入messages
- `qianfan.trainer.LLMFinetune` 大语言模型fine-tune任务Trainer实现
- `qianfan.dataset.Dataset` 千帆dataset类，用于管理千帆平台、本地、第三方数据集的导入导出，数据清洗等操作

In [ ]:
from qianfan.trainer.consts import ActionState
from qianfan.model.consts import ServiceType
from qianfan.resources.console import consts as console_consts
from qianfan.trainer.configs import TrainConfig
from qianfan.model.configs import DeployConfig
from qianfan.resources import QfMessages
from qianfan.trainer import LLMFinetune
from qianfan.dataset import Dataset
from qianfan.utils import enable_log
import logging

enable_log(logging.INFO)

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地和千帆平台。

In [3]:
from qianfan.dataset import Dataset

# 加载本地数据集
ds: Dataset = Dataset.load(data_file="./data/fin_cqa_train.jsonl")
ds.list()

[[{'prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的产品是？在国际奶粉价格下降压力下,国内奶价仍有下降空间',
   'response': [['奶']]}],
 [{'prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的地区是？在国际奶粉价格下降压力下,国内奶价仍有下降空间',
   'response': [['国内']]}],
 [{'prompt': '下文中有哪些因果事件？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['需求增加导致市场价格提升']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的行业是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['我们无法得知，可能需要更多内容说明。']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']]}],
 [{'prompt': '下文中有哪些因果事件？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌',
   'response': [['市场价格下降导致市场价格下降']]}],
 [{'prompt': '下文中市场价格下降导致市场价格下降事件对应的原因涉及的产品是？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌',
   'response': [['7-ACA']]}],
 [{'prompt': '下文中市场价格下降导致市场价

从本地数据集上传到BOS

In [ ]:
# 保存到千帆平台
from qianfan.dataset.data_source import QianfanDataSource
from qianfan.resources.console import consts as console_consts

bos_bucket_name = "sdk-test"
bos_bucket_file_path = "/sdk_ds/"


# 创建千帆数据集，并上传保存
qianfan_data_source = QianfanDataSource.create_bare_dataset(
    name="sdk_trainer_ds_022z",
    template_type=console_consts.DataTemplateType.NonSortedConversation,
    storage_type=console_consts.DataStorageType.PrivateBos,
    storage_id=bos_bucket_name,
    storage_path=bos_bucket_file_path,
)

ds.save(qianfan_data_source, replace_source=True)

[INFO] [12-08 18:23:51] data_source.py:749 [t:140487811458880]: start to create dataset on qianfan
[INFO] [12-08 18:23:52] data_source.py:767 [t:140487811458880]: create dataset on qianfan successfully
[INFO] [12-08 18:23:52] schema.py:30 [t:140487811458880]: unpack dataset before validating
[INFO] [12-08 18:23:52] dataset.py:817 [t:140487811458880]: list local dataset data by 0
[INFO] [12-08 18:23:52] schema.py:33 [t:140487811458880]: pack dataset after validation
[INFO] [12-08 18:23:52] dataset.py:171 [t:140487811458880]: export as format: FormatType.Jsonl
[INFO] [12-08 18:23:52] dataset.py:183 [t:140487811458880]: enter packed deserialization logic
[INFO] [12-08 18:23:52] data_source.py:448 [t:140487811458880]: start to upload data to user BOS
[INFO] [12-08 18:23:52] data_source.py:461 [t:140487811458880]: uploading data to user BOS finished
[INFO] [12-08 18:23:54] data_source.py:294 [t:140487811458880]: successfully create importing task
[INFO] [12-08 18:23:56] data_source.py:297 [

True

### LLMFinetune 训练
`LLMFinetune` 实现了SFT逻辑的trainer，它内部组装了SFT所需要的基本`Pipeline`, 用于串联数据->训练->模型发布->服务调用等步骤

In [6]:
from qianfan.trainer.consts import PeftType

trainer = LLMFinetune(
    train_type="ERNIE-Speed",
    train_config=TrainConfig(
        epoch=1,
        learning_rate=0.0003,
        max_seq_len=4096,
        peft_type=PeftType.LoRA,
        logging_steps=1,
        warmup_ratio=0.10,
        weight_decay=0.0100,
        lora_rank=8,
        lora_all_linear="True",
    ),
    dataset=ds,
)

### 运行任务
同步运行trainer，训练直到模型发布完成

In [7]:
trainer.run()

[INFO] [12-08 11:40:06] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:12] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:15] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:18] data_source.py:1044 [t:140481718310720]: data releasing, keep rolling
[INFO] [12-08 11:40:22] data_source.py:1053 [t:140481718310720]: data releasing succeeded
[INFO] [12-08 11:40:25] actions.py:352 [t:140481718310720]: [train_action] fine-tune running... current status: RUNNING, check vdl report in https://console.bce.baidu.com/qianfan/visualdl/index?displayToken=eyJydW5JZCI6InJ1bi1kdjlqYjNnMW13NWkxdWRqIn0=
[INFO] [12-08 11:40:56] actions.py:352 [t:140481718310720]: [train_action] fine-tune running... current status: RUNNING, check vdl report in https://console.bce.baidu.com/qianfan/visualdl/index?displayToken=eyJydW5JZCI6InJ1bi1kdjlqYjNnMW13NWkxdWRqIn0=
[INFO] [12-08 11:41:26] action

获取finetune任务输出：

In [8]:
trainer.output

{'task_id': 17320,
 'job_id': 9098,
 'model_id': 'am-cmyddmcb0b1e',
 'model_version_id': 'amv-7598fwf39uf2',
 'model': <qianfan.trainer.model.Model at 0x7fc40ac23ad0>}

In [3]:
o["model_id"]

'10268'

### 运行批量评估推理
Model支持模型批量运行评估数据集，并保存到千帆平台

调用Fine-tune得到的Model对象的`batch_run_on_qianfan`发起批量任务，这可能会持续数十分钟

In [17]:
from qianfan.model import Model
from qianfan.dataset import Dataset

# 首先需要先加载测试数据集，这里以加载平台预置数据集为例子：
test_ds = Dataset.load(qianfan_dataset_id="xxx", is_download_to_local=False)

# 从训练结果中获取模型对象
m: Model = trainer.output["model"]

# 运行批量任务获取结果数据集
result_ds: Dataset = m.batch_inference(test_ds)

[INFO] [12-08 12:22:22] model.py:295 [t:140481718310720]: start to create evaluation task in model
[INFO] [12-08 12:22:25] model.py:319 [t:140481718310720]: start to polling status of evaluation task 2419
[INFO] [12-08 12:22:26] model.py:326 [t:140481718310720]: current eval_state: Doing
[INFO] [12-08 12:22:57] model.py:326 [t:140481718310720]: current eval_state: Doing
[INFO] [12-08 12:23:27] model.py:326 [t:140481718310720]: current eval_state: Doing
[INFO] [12-08 12:38:42] model.py:326 [t:140481718310720]: current eval_state: DoingWithManualBegin
[INFO] [12-08 12:38:42] model.py:344 [t:140481718310720]: get result dataset id 39815
[INFO] [12-08 12:38:42] dataset.py:353 [t:140481718310720]: no data source was provided, construct
[INFO] [12-08 12:38:42] dataset.py:255 [t:140481718310720]: construct a qianfan data source from existed id: 39815, with args: {}
[INFO] [12-08 12:38:43] data_source.py:1022 [t:140481718310720]: start to fetch dataset cache because is_download_to_local is set

通过这种方式运行完成后，可以直接在本地拿到一份批量运行的结果，我们可以通过dataset.list查看其中的部分数据：

In [ ]:
result_ds.list([i for i in range(3,6)])

[[{'model_response': [{'content': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是高碳铬铁。',
     'model_version_id': 12722,
     'tag': 'm_17320_9098>V1'}],
   'prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']],
   'source_entity_id': '2d469d3fb6dea6e5ca6a1c7c0e91dceee6f918b495e22bedd1b496fc3f92147f'}],
 [{'model_response': [{'content': '根据上下文，需求增加导致市场价格上涨的事件涉及的产品是高碳铬铁。',
     'model_version_id': 12722,
     'tag': 'm_17320_9098>V1'}],
   'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']],
   'source_entity_id': 'b7e4ea8632312142d853344214d34256d20a10d9fb890a81417a2ea9d2e70f04'}],
 [{'model_response': [{'content': '- 产品：煤炭\n- 原因：煤炭价格大幅上涨',
     'model_version_id': 12722,
     'tag': 'm_17320_9098>V1'}],
   'prompt': '下文中市场价格提升导致运营成本提升事件对应的原因涉及的产品是？利润和每股收益大幅下降的主要原因是煤炭价格大幅上涨，导致公司投资的火电企业的煤炭成本大幅增加###但报告期内受煤炭行业去产能改革的影响，公司煤炭前三季度平均采购价格同比上涨44.53%，导致成本涨幅抵消了销售价

在完成模型的批量运行后，我们可以对模型有一个简单的体感评估，如果效果不错，我们可以选择发布成服务以最终应用生产：

In [18]:
from qianfan.model import Service
from qianfan.model.consts import ServiceType
from qianfan.resources.console.consts import DeployPoolType

sft_svc: Service = m.deploy(DeployConfig(
    name="sdkcqasvc",
    endpoint_prefix="sdkcqa1",
    replicas=1, # 副本数， 与qps强绑定
    pool_type=DeployPoolType.PrivateResource, # 私有资源池
    service_type=ServiceType.Chat,
))


[INFO] [12-08 12:40:10] model.py:497 [t:140481718310720]: ready to deploy service with model 10268/12722
[INFO] [12-08 12:40:18] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:40:52] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:41:22] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:41:53] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deployment payment.
[INFO] [12-08 12:42:24] model.py:529 [t:140481718310720]: please check web console `https://console.bce.baidu.com/qianfan/ais/console/onlineService`,for service  deplo

使用Finetune之后的模型服务和原始的预置模型服务调用：

In [26]:
from qianfan import ChatCompletion
### 使用Model & Service调用模型

problem="下文中有哪些因果事件？无取向硅钢广泛应用于铁芯等电机零部件其产量的持续提升导致市场竞争愈发激烈，价格进一步降低，从而有效减少新能源汽车驱动电机行业内企业的成本支出"

#获取服务对象，即ChatCompletion等类型的对象
chat_comp: ChatCompletion = sft_svc.get_res()
sft_chat_resp = chat_comp.do(messages=[{"content": problem, "role": "user"}])
sft_chat_resp["result"]

[INFO] [12-08 13:30:55] openapi_requestor.py:134 [t:140481718310720]: requesting llm api endpoint: /chat/ex1ndqpo_sdkcqa1


'因果事件如下：\n- 因果事件：无取向硅钢产量的持续提升导致市场竞争愈发激烈，价格进一步降低，从而有效减少新能源汽车驱动电机行业内企业的成本支出'